In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random

In [210]:
data = pd.read_csv('dataset/dataCovidIndonesia.csv')
data = data.loc[0:49, 'Kasus harian']
data = np.genfromtxt(data, dtype= None)
data

array([  2,   0,   0,   0,   2,   0,   2,  13,   8,   0,  35,  27,  21,
        17,  38,  55,  82,  60,  81,  64,  65, 106, 105, 103, 153, 109,
       130, 129, 114, 149, 113, 196, 106, 181, 218, 247, 218, 337, 219,
       330, 399, 316, 282, 297, 380, 407, 325, 327, 185, 375])

In [211]:
def generasi_awal(pop_size, gene_per_pop):
    population_size = (pop_size, gene_per_pop)
    gene = np.random.uniform(low=0.1, high=0.9, size=population_size)
    return gene #generate individu dengan 3 gen pada setiap individu

In [212]:
def day_predict(pop_size, gene_per_pop):
    day = []
    for i in range(2, pop_size):
        row = []
        for j in range(0, gene_per_pop):
            if j == 0:
                row.append(data[i-1]) #y(t-1) 
            elif j == 1:
                row.append(data[i-2]) #y(t-2)
            elif j == 2:
                row.append(data[i]) #y(target)
        day.append(row)
    return day

In [213]:
def y_pred(pop_size, gene, day_predict, gene_per_pop):
    y_pred_indv = []
    for i in range (0,pop_size-2):
        row = []
        for j in gene:
            total = 0
            z = 0
            for k in j:
                if k<gene_per_pop-1:
                    g = k*day_predict[i][z]
                else:
                    g = k
                z=z+1
                total = total + g
            row.append(total)
        y_pred_indv.append(row)
    return y_pred_indv #generate y_pred_individu

In [214]:
def fitness_func(pop_size, day_predict, hasil_prediksi, gene_per_pop):  
    mape = []
    fitn = []
    for i in range(0,pop_size):
        row = []
        for j in range(0,len(data_y)):
            if hasil_prediksi[j][i]>0: #hitung mape
                row.append(abs((hasil_prediksi[j][i]-data_y[j])/hasil_prediksi[j][i]))
            else:
                row.append(abs(0)) #mape akan bernilai 0 ketika hasil_prediksi bernilai 0
        mape.append(row)        
    for i in range(0,len(mape)):
        f = 0
        for j in range (0,pop_size-2):
            f = f + mape[i][j]
        f=f/len(mape)
        fitn.append(1/(f+0.01))
    
    return fitn

In [215]:
def probability(fitness):
    prob = []
    sumF = 0
    for i in range(0,len(fitness)):
        sumF = sumF + fitness[i]
    for i in range(0,len(fitness)):
        prob.append(fitness[i]/sumF*100)
    return prob #generate probabilitas setiap fitness individu

In [216]:
def rangePc(probs):
    ranges = []
    x = 0
    y = 0
    minArray = []
    maxArray = []
    for i in range(0,len(probs)):
        if i == 0:
            x = 0;
        else:
            x = x + probs[i-1]
        y = y + probs[i]
        minArray.append(x)
        maxArray.append(y)
    ranges = np.vstack((minArray, maxArray)).T
    return ranges #generate nilai min dan max pada setiap probabilitas indifidu

In [217]:
def round_wheel(pc,fitness,ranges):
    rW = []
    for i in range(0,int(pc*50)):
        rNum = random.uniform(0,100)
        for j in range(0,len(ranges)):
            if rNum >= ranges[j][0] and rNum <= ranges[j][1]:
                rW.append(j)
    return rW #terpilih individu ke-sekian

In [218]:
def cross_over(rw,gA):
    co = []
    x = 0
    y = 0
    for i in range(0,len(rw)):
        row = []
        for j in range(0,gene_per_pop):
            row.append(gA[rw[i]][j])
        co.append(row) #mendapatkan nilai gen dari hasil roundwheel
    
    for i in range(0,len(rw),2):
        if i % 2 == 0 and i == len(rw)-1:
            co[i][1], co[0][1] = co[0][1], co[i][1] #swap pasangan akhir ke pasangan awal ketika jumlah datanya ganjil
        else:
            co[i][1], co[i+1][1] = co[i+1][1], co[i][1]
    return co #menghasilkan nilai offspring berdasarkan hasil crossover

In [219]:
pop_size = 50
gene_per_pop = 3
pc = 0.7
generasiAwal = generasi_awal(pop_size, gene_per_pop)
generasiAwal

array([[0.23813379, 0.76336748, 0.51770668],
       [0.89995175, 0.65447037, 0.16206945],
       [0.14239832, 0.6947438 , 0.67092363],
       [0.46953745, 0.79731594, 0.52909758],
       [0.26669189, 0.89380593, 0.22439752],
       [0.44231541, 0.87264597, 0.37566842],
       [0.57934368, 0.28095341, 0.60650748],
       [0.56819644, 0.43191763, 0.42260352],
       [0.24255344, 0.33128584, 0.12798989],
       [0.86341133, 0.83484345, 0.28140606],
       [0.59817052, 0.27314604, 0.66407172],
       [0.67926788, 0.26585353, 0.87132549],
       [0.17852716, 0.75995258, 0.46322621],
       [0.29446599, 0.7627028 , 0.32816367],
       [0.52409613, 0.29431523, 0.16224531],
       [0.52156369, 0.4229955 , 0.77875672],
       [0.18416372, 0.34756963, 0.48238451],
       [0.32241395, 0.8228891 , 0.13180291],
       [0.82527172, 0.27681003, 0.5233844 ],
       [0.41840442, 0.1817988 , 0.66181954],
       [0.66297229, 0.60126374, 0.57849625],
       [0.86992625, 0.47868121, 0.23174952],
       [0.

In [220]:
dayPredict = day_predict(pop_size, gene_per_pop)
dayPredict

[[0, 2, 0],
 [0, 0, 0],
 [0, 0, 2],
 [2, 0, 0],
 [0, 2, 2],
 [2, 0, 13],
 [13, 2, 8],
 [8, 13, 0],
 [0, 8, 35],
 [35, 0, 27],
 [27, 35, 21],
 [21, 27, 17],
 [17, 21, 38],
 [38, 17, 55],
 [55, 38, 82],
 [82, 55, 60],
 [60, 82, 81],
 [81, 60, 64],
 [64, 81, 65],
 [65, 64, 106],
 [106, 65, 105],
 [105, 106, 103],
 [103, 105, 153],
 [153, 103, 109],
 [109, 153, 130],
 [130, 109, 129],
 [129, 130, 114],
 [114, 129, 149],
 [149, 114, 113],
 [113, 149, 196],
 [196, 113, 106],
 [106, 196, 181],
 [181, 106, 218],
 [218, 181, 247],
 [247, 218, 218],
 [218, 247, 337],
 [337, 218, 219],
 [219, 337, 330],
 [330, 219, 399],
 [399, 330, 316],
 [316, 399, 282],
 [282, 316, 297],
 [297, 282, 380],
 [380, 297, 407],
 [407, 380, 325],
 [325, 407, 327],
 [327, 325, 185],
 [185, 327, 375]]

In [221]:
yPred = y_pred(pop_size, generasiAwal, dayPredict, gene_per_pop)
yPred

[[1.5267349571011453,
  1.308940733685303,
  1.3894875902417265,
  1.5946318738339123,
  1.7876118557758518,
  1.7452919370283977,
  0.5619068138353424,
  0.8638352512525971,
  0.6625716898577796,
  1.6696868911119347,
  0.5462920839999201,
  0.5317070644556814,
  1.5199051543858069,
  1.525405596568514,
  0.5886304581516262,
  0.8459910025158259,
  0.6951392536753822,
  1.6457781905407483,
  0.5536200500987438,
  0.363597601335416,
  1.2025274793682097,
  0.957362421919483,
  1.6345572258492582,
  0.5573073965146194,
  0.9074206761098624,
  1.0490441369437213,
  1.1786031321586914,
  1.2731763886610228,
  0.8481620861915495,
  0.6075087789539149,
  0.6993489740212033,
  0.7773828122324085,
  1.6363579618610726,
  0.20516074924718541,
  1.6051096409264076,
  0.9922504542050472,
  1.4362429065671622,
  1.655056990398447,
  1.189497408925891,
  0.9258351265565039,
  0.7456669117652464,
  1.6447528053269387,
  0.6876204313498822,
  1.5970232237928355,
  0.23704320834566542,
  1.6224699822

In [222]:
dayPredict = np.array(dayPredict)
data_y = dayPredict[:, (gene_per_pop-1)]
fitness = fitness_func(pop_size, day_predict, yPred, gene_per_pop)
fitness

[2.8416410376993926,
 1.7965544610435977,
 3.0153015076199705,
 2.3302213451993796,
 2.282955716215746,
 2.3501737321729936,
 2.98490385727087,
 2.907759892672052,
 1.142489877963761,
 1.8951756138792297,
 2.8296384259540814,
 2.3681691193845866,
 3.0323883067217365,
 2.7493451540374574,
 1.9430361302604857,
 2.5000680869498626,
 4.137628354226999,
 1.8564467765951609,
 2.499510736631685,
 4.006285943092171,
 2.2666970039567205,
 2.1170416485788817,
 1.997856962441821,
 2.740192370761664,
 3.291806832582755,
 2.623448833187862,
 2.1191960715367886,
 2.0154382569563576,
 2.87094104357933,
 1.6985366881429347,
 1.8543117178800084,
 2.316934476832492,
 1.986745464633473,
 1.7480732975854176,
 1.9034519197912532,
 2.19051005086725,
 1.7822288766366183,
 1.85263325568243,
 2.0425918896980257,
 4.224192362630606,
 1.63402197847507,
 2.2722359106349375,
 3.5812959154887127,
 2.11650180563198,
 2.591137537219049,
 2.2131193294127867,
 2.896008524893917,
 2.553887062856458,
 2.093174253016938,


In [223]:
yProb = probability(fitness)
yProb

[2.3400564072533947,
 1.4794404788538498,
 2.4830636660637078,
 1.918908587260229,
 1.8799859237433905,
 1.9353391322721367,
 2.458031575267873,
 2.394504470244592,
 0.9408263477626716,
 1.5606537839553896,
 2.3301724042615426,
 1.9501581120752958,
 2.497134434745283,
 2.2640518834374372,
 1.6000663299197058,
 2.0587752878784427,
 3.4072860057585017,
 1.528761063295136,
 2.0583163167537175,
 3.299127147323308,
 1.866597074381113,
 1.7433577318372635,
 1.645210610247511,
 2.256514679102134,
 2.7107623967389554,
 2.1603778132979383,
 1.7451318726170808,
 1.6596885898130995,
 2.3641846013434615,
 1.3987240496997293,
 1.527002868730808,
 1.9079670147527317,
 1.6360604286110372,
 1.4395168376633136,
 1.5674692199731277,
 1.8038580565527156,
 1.4676435364763771,
 1.5256206757769242,
 1.6820492720507088,
 3.4785704008721066,
 1.345596980616224,
 1.871158295833642,
 2.949153092219775,
 1.742913177817228,
 2.133769858895754,
 1.8224765190615104,
 2.3848273635663055,
 2.1030945518992086,
 1.7237

In [224]:
yRanges = rangePc(yProb)
yRanges

array([[  0.        ,   2.34005641],
       [  2.34005641,   3.81949689],
       [  3.81949689,   6.30256055],
       [  6.30256055,   8.22146914],
       [  8.22146914,  10.10145506],
       [ 10.10145506,  12.0367942 ],
       [ 12.0367942 ,  14.49482577],
       [ 14.49482577,  16.88933024],
       [ 16.88933024,  17.83015659],
       [ 17.83015659,  19.39081037],
       [ 19.39081037,  21.72098278],
       [ 21.72098278,  23.67114089],
       [ 23.67114089,  26.16827532],
       [ 26.16827532,  28.43232721],
       [ 28.43232721,  30.03239354],
       [ 30.03239354,  32.09116882],
       [ 32.09116882,  35.49845483],
       [ 35.49845483,  37.02721589],
       [ 37.02721589,  39.08553221],
       [ 39.08553221,  42.38465936],
       [ 42.38465936,  44.25125643],
       [ 44.25125643,  45.99461416],
       [ 45.99461416,  47.63982477],
       [ 47.63982477,  49.89633945],
       [ 49.89633945,  52.60710185],
       [ 52.60710185,  54.76747966],
       [ 54.76747966,  56.51261154],
 

In [229]:
roundWheel = round_wheel(pc,fitness,yRanges)
roundWheel

[12,
 2,
 26,
 17,
 42,
 12,
 2,
 3,
 10,
 2,
 8,
 22,
 16,
 29,
 7,
 34,
 21,
 0,
 43,
 4,
 39,
 31,
 26,
 35,
 48,
 49,
 5,
 34,
 18,
 10,
 24,
 42,
 18,
 19,
 2]

In [230]:
crossOver = cross_over(roundWheel,generasiAwal)
crossOver

[[0.17852716275132066, 0.6947437951208633, 0.4632262055704648],
 [0.1423983196974927, 0.7599525771929034, 0.6709236345454613],
 [0.8498865702434598, 0.8228890952703741, 0.49421392877810677],
 [0.3224139545640907, 0.5893015660793457, 0.131802908709998],
 [0.44556977721655666, 0.7599525771929034, 0.4859539117498056],
 [0.17852716275132066, 0.3438102156749411, 0.4632262055704648],
 [0.1423983196974927, 0.7973159369169561, 0.6709236345454613],
 [0.4695374498848217, 0.6947437951208633, 0.5290975767580584],
 [0.5981705211376267, 0.6947437951208633, 0.6640717214711145],
 [0.1423983196974927, 0.27314604199996007, 0.6709236345454613],
 [0.24255344025198777, 0.8172786129246291, 0.12798989157658636],
 [0.7708228440301416, 0.3312858449288898, 0.5050466710723357],
 [0.18416371848142124, 0.30375438947695743, 0.48238451034251373],
 [0.5221176338143109, 0.3475696268376911, 0.1302988822943097],
 [0.5681964419328287, 0.8025548204632038, 0.42260352002944224],
 [0.8316970440411161, 0.43191762562629854, 0.